In [ ]:
import os
import sys
cwd = os.getcwd()
source_path = os.path.join(cwd, '..')
print(source_path)
sys.path.append(source_path)

import logging
logging.getLogger('numba').setLevel(logging.WARNING)
logging.getLogger('matplotlib').setLevel(logging.WARNING)

import numpy as np
import dask.array as da
from joblib import Parallel, delayed
import matplotlib.pyplot as plt

from src import Data, Parameters, Pycro, DownloadData, SimpleCellposeSegmentaion, BIGFISH_SpotDetection, CellProperties, Save_Images, Save_Masks, Save_Outputs, Save_Parameters, remove_local_data, return_to_NAS


In [ ]:
params = Parameters()
data = Data('DEFAULT_NAME')

params.num_chunks_to_run = 2

In [ ]:
params.load_in_mask = True
params.initial_data_location = None
params.independent_params = {}

if data.images is None:
    params.local_dataset_location = DownloadData().main(**params.__dict__)['local_dataset_location']
    output = Pycro().main(**params.__dict__)
    data.append(output)

pp = data.images.shape[0]
tt = data.images.shape[1]
cc = data.images.shape[2]
zz = data.images.shape[3]
yy = data.images.shape[4]
xx = data.images.shape[5]

print(data)

In [ ]:
data.nuc_masks = da.zeros([pp, tt, yy, xx])
params.nucChannel = 2


def process_chunk(p, t, params):
    output = SimpleCellposeSegmentaion().main(**params)
    return p, t, output['nuc_mask']

futures = []
for p in range(pp):
    for t in range(tt):
        if len(futures) >= params.num_chunks_to_run:
            break
        futures.append((p, t, 
                        {**params.__dict__, 
                         **data.__dict__, 
                         'image': data.images[p, t], 
                         'fov': p, 
                         'timepoint': t}))

results = Parallel(n_jobs=-1)(
    delayed(process_chunk)(p, t, params) for p, t, params in futures
)

for p, t, nuc_mask in results:
    data.nuc_masks[p, t, :, :] = nuc_mask


In [ ]:
plt.imshow(data.nuc_masks[0, 0, :, :])
plt.show()

In [ ]:
params.FISHChannel = [0]
params.bigfish_alpha = 0.99
params.bigfish_beta = 1
params.bigfish_gamma = 5
params.voxel_size_yx = 130
params.voxel_size_z = 500
params.spot_yx = 160
params.spot_z = 1000

def process_chunk(p, t, params):
    output = BIGFISH_SpotDetection().main(**params)
    return p, t, output

futures = []
for p in range(pp):
    for t in range(tt):
        if len(futures) >= params.num_chunks_to_run:
            break
        kwargs = (p, t, 
                  {**params.__dict__, 
                   **data.__dict__, 
                   'image': data.images[p, t], 
                   'fov': p, 
                   'timepoint': t,
                   'nuc_mask': np.array(data.nuc_masks[p,t]) if data.nuc_masks is not None else None,
                   'cell_mask': np.array(data.cell_masks[p, t]) if data.cell_masks is not None else None})
        futures.append(kwargs)

results = Parallel(n_jobs=-1)(
    delayed(process_chunk)(p, t, params) for p, t, params in futures
)

for p, t, output in results:
    data.append(output)


In [ ]:
def process_chunk(p, t, params):
    output = CellProperties().main(**params)
    return p, t, output

futures = []
for p in range(pp):
    for t in range(tt):
        if len(futures) >= params.num_chunks_to_run:
            break
        kwargs = (p, t, 
                  {**params.__dict__, 
                   **data.__dict__, 
                   'image': data.images[p, t], 
                   'fov': p, 
                   'timepoint': t,
                   'nuc_mask': np.array(data.nuc_masks[p,t][np.newaxis, :, :]) if data.nuc_masks is not None else None,
                   'cell_mask': np.array(data.cell_masks[p, t][np.newaxis, :, :]) if data.cell_masks is not None else None})
        futures.append(kwargs)

results = Parallel(n_jobs=-1)(
    delayed(process_chunk)(p, t, params) for p, t, params in futures
)

for p, t, output in results:
    data.append(output)

In [ ]:
Save_Images().main(**{**params.__dict__, **data.__dict__})

Save_Masks().main(**{**params.__dict__, **data.__dict__, 
                    'masks':{'nuc_masks': data.nuc_masks}})

Save_Parameters().main(params, **{**params.__dict__, **data.__dict__})

Save_Outputs().main(data.__dict__, **{**params.__dict__, 'position_indexs': data.position_indexs})


In [ ]:
return_to_NAS().main(**params.__dict__)

In [ ]:
print(params)
print(data)